In [1]:
from matplotlib import pyplot as plt
import xarray as xr
import numpy as np
import dask
from dask.diagnostics import progress
from tqdm.autonotebook import tqdm 
import intake
import fsspec
import seaborn as sns

%matplotlib inline
%config InlineBackend.figure_format = 'retina' 

<ipython-input-1-d0d7acd7e5d4>:6: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
col = intake.open_esm_datastore("https://storage.googleapis.com/cmip6/pangeo-cmip6.json")

In [3]:
[eid for eid in col.df['experiment_id'].unique() if 'ssp' in eid]

['ssp370SST-ssp126Lu',
 'ssp370SST-lowNTCF',
 'ssp370SST-lowCH4',
 'ssp370-lowNTCF',
 'ssp370pdSST',
 'ssp370SST',
 'ssp245',
 'esm-ssp585',
 'ssp126-ssp370Lu',
 'ssp370-ssp126Lu',
 'esm-ssp585-ssp126Lu',
 'ssp585',
 'ssp370',
 'ssp126',
 'ssp119',
 'ssp245-GHG',
 'ssp245-nat',
 'ssp434',
 'ssp460',
 'ssp534-over',
 'ssp245-aer',
 'ssp245-stratO3',
 'ssp245-cov-fossil',
 'ssp245-covid',
 'ssp245-cov-strgreen',
 'ssp245-cov-modgreen',
 'ssp585-bgc']

In [11]:
# there is currently a significant amount of data for these runs
#expts = ['historical', 'ssp245', 'ssp585']
expts = ['historical','ssp126','ssp245','ssp585']
query = dict(
    experiment_id=expts,
    table_id='day',                           
    variable_id=['tas'],
    member_id = 'r1i1p1f1',                     
)

col_subset = col.search(require_all_on=["source_id"], **query)
col_subset.df.groupby("source_id")[
    ["experiment_id", "variable_id", "table_id"]
].nunique()

,experiment_id,variable_id,table_id
source_id,,,
ACCESS-CM2,4,1,1
ACCESS-ESM1-5,4,1,1
AWI-CM-1-1-MR,4,1,1
BCC-CSM2-MR,4,1,1
CESM2-WACCM,4,1,1
CMCC-CM2-SR5,4,1,1
CMCC-ESM2,4,1,1
CanESM5,4,1,1
EC-Earth3,4,1,1


In [12]:
#ds = intake.open_esm_datastore('/Volumes/Transcend/tas_Amon_CAS-ESM2-0_ssp245_r1i1p1f1_gn_201501-210012.nc')
def drop_all_bounds(ds):
    drop_vars = [vname for vname in ds.coords
                 if (('_bounds') in vname ) or ('_bnds') in vname]
    return ds.drop(drop_vars)

def open_dset(df):
    assert len(df) == 1
    ds = xr.open_zarr(fsspec.get_mapper(df.zstore.values[0]), consolidated=True)
    return drop_all_bounds(ds)

def open_delayed(df):
    return dask.delayed(open_dset)(df)

from collections import defaultdict
dsets = defaultdict(dict) 

for group, df in col_subset.df.groupby(by=['source_id', 'experiment_id']):
    dsets[group[0]][group[1]] = open_delayed(df)

In [13]:
dsets_ = dask.compute(dict(dsets))[0]

/home/lulur/.conda/envs/cicoes39/lib/python3.9/site-packages/xarray/coding/times.py:527: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/home/lulur/.conda/envs/cicoes39/lib/python3.9/site-packages/xarray/coding/times.py:527: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/home/lulur/.conda/envs/cicoes39/lib/python3.9/site-packages/numpy/core/_asarray.py:102: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return array(a, dtype, copy=False, order=order)
/home/lulur/.conda/envs/cicoes39/lib/python3.

In [14]:
# calculate global means

def get_lat_name(ds):
    for lat_name in ['lat', 'latitude']:
        if lat_name in ds.coords:
            return lat_name
    raise RuntimeError("Couldn't find a latitude coordinate")

def global_mean(ds):
    lat = ds[get_lat_name(ds)]
    weight = np.cos(np.deg2rad(lat))
    weight /= weight.mean()
    other_dims = set(ds.dims) - {'time'}
    return (ds * weight).mean(other_dims)

In [ ]:

for k, v in tqdm(dsets_.items()):

    # Check for any missing experiments that we expect
    expt_dsets = v.values()
    if any([d is None for d in expt_dsets]):
        print(f"Missing experiment for {k}")
        continue

    climatology = v['historical'].sel(time=slice('1981-01-01', '2010-12-31')).groupby('time.month').mean('time')

    for i in v:
        if i == 'historical':
            # When working with daily data, it's very easy for objects to run your computer out of memory, so we shorten historical runs because to the date range we need.
            anomaly = v[i].sel(time=slice('1981-01-01', '2010-12-31')).groupby('time.month') - climatology
        else:
            anomaly = v[i].groupby('time.month') - climatology
        # Because these files are too large to store in memory, we use the option compute=False to create a dask delayed object and then compute it later.
        # Remember to change the file location to the relevant folder.
        delayed_obj = anomaly.to_netcdf(path=f"/nobackup/users/lulur/tas-anomaly_{k}_{i}.nc", mode='w', compute=False, engine='netcdf4', format='NETCDF4')
        print(f"writing data to /nobackup/users/lulur/tas-anomaly_{k}_{i}.nc")

        with progress.ProgressBar():
            results = delayed_obj.compute()

In [19]:
#for a SPECIFIC model, use the cell below:
# Check for any missing experiments that we expect
models = ['ACCESS-ESM1-5','NorESM2-LM','NorESM2-MM']
for k in models:
#k = 'ACCESS-WACCM'
    v = dsets_[k]
    expt_dsets = v.values()
    #if any([d is None for d in expt_dsets]):
        #print(f"Missing experiment for {k}")
        #continue

    climatology = v['historical'].sel(time=slice('1981-01-01', '2010-12-31')).groupby('time.dayofyear').mean('time')

    for i in v:
        if i == 'historical':
            # When working with daily data, it's very easy for objects to run your computer out of memory, so we shorten historical runs because to the date range we need.
            anomaly = v[i].sel(time=slice('1981-01-01', '2010-12-31')).groupby('time.dayofyear') - climatology
        else:
            anomaly = v[i].groupby('time.dayofyear') - climatology
        # Because these files are too large to store in memory, we use the option compute=False to create a dask delayed object and then compute it later.
        # Remember to change the file location to the relevant folder.
        delayed_obj = anomaly.to_netcdf(path=f"/nobackup/users/lulur/tas-anomaly_{k}_{i}.nc", mode='w', compute=False, engine='netcdf4', format='NETCDF4')
        print(f"writing data to /nobackup/users/lulur/tas-anomaly_{k}_{i}.nc")

        with progress.ProgressBar():
            results = delayed_obj.compute()

writing data to /nobackup/users/lulur/tas-anomaly_ACCESS-ESM1-5_historical.nc
[########################################] | 100% Completed | 27.3s
writing data to /nobackup/users/lulur/tas-anomaly_ACCESS-ESM1-5_ssp126.nc
[########################################] | 100% Completed |  3min  6.9s
writing data to /nobackup/users/lulur/tas-anomaly_ACCESS-ESM1-5_ssp245.nc
[########################################] | 100% Completed |  3min 40.2s


ValueError: Unable to update size for existing dimension'time' (104459 != 10957)

In [ ]:
expt_da = xr.DataArray(expts, dims='experiment_id', name='experiment_id',
                        coords={'experiment_id': expts})

dsets_aligned = {}

for k, v in tqdm(dsets_.items()):
    expt_dsets = v.values()
    if any([d is None for d in expt_dsets]):
        print(f"Missing experiment for {k}")
        continue
    
    
    print(type(v[expt]) for expt in expts)
    for ds in expt_dsets:
        #ds.coords['year'] = ds.time.dt.year
        #ds.coords['month']=ds.time.dt.month
        #ds.coords['rawday'] = cftime.date2num
        ds.coords['rawmonth']=ds.time.dt.month+(ds.time.dt.year-1850)*12
        #print(ds.coords)
    # workaround for
    # https://github.com/pydata/xarray/issues/2237#issuecomment-620961663
    dsets_mon_mean = [v[expt].pipe(global_mean)
                             .swap_dims({'time': 'rawmonth'})
                             .drop('time')
                             #.drop('mon')
    #                         .coarsen(year=12).mean()
                      for expt in expts]
    
    #print(dsets_mon_mean)
    
    # align everything with the 4xCO2 experiment
    dsets_aligned[k] = xr.concat(dsets_mon_mean, join='outer',
                                 dim=expt_da)

In [ ]:
# expt_da = xr.DataArray(expts, dims='experiment_id', name='experiment_id',
#                         coords={'experiment_id': expts})

# dsets_aligned = {}

# for k, v in tqdm(dsets_.items()):
#     expt_dsets = v.values()
#     if any([d is None for d in expt_dsets]):
#         print(f"Missing experiment for {k}")
#         continue
    
#     for ds in expt_dsets:
#         #ds.coords['year'] = ds.time.dt.year
#         #ds.coords['month']=ds.time.dt.month
#         ds.coords['rawmonth']=ds.time.dt.month+(ds.time.dt.year-1850)*12
#         #print(ds.coords)
#     # workaround for
#     # https://github.com/pydata/xarray/issues/2237#issuecomment-620961663
#     dsets_mon_mean = [v[expt].pipe(global_mean)
#                              .swap_dims({'time': 'rawmonth'})
#                              .drop('time')
#                              #.drop('mon')
#     #                         .coarsen(year=12).mean()
#                       for expt in expts]
    
#     #print(dsets_mon_mean)
    
#     # align everything with the 4xCO2 experiment
#     dsets_aligned[k] = xr.concat(dsets_mon_mean, join='outer',
#                                  dim=expt_da)

In [ ]:
# expt_da = xr.DataArray(expts, dims='experiment_id', name='experiment_id',
#                        coords={'experiment_id': expts})

# dsets_aligned = {}

# for k, v in tqdm(dsets_.items()):
#     expt_dsets = v.values()
#     if any([d is None for d in expt_dsets]):
#         print(f"Missing experiment for {k}")
#         continue
    
#     for ds in expt_dsets:
#         ds.coords['year'] = ds.time.dt.year
        
#     # workaround for
#     # https://github.com/pydata/xarray/issues/2237#issuecomment-620961663
#     dsets_ann_mean = [v[expt].pipe(global_mean)
#                              .swap_dims({'time': 'year'})
#                              .drop('time')
#                              .coarsen(year=12).mean()
#                       for expt in expts]
    
#     # align everything with the 4xCO2 experiment
#     dsets_aligned[k] = xr.concat(dsets_ann_mean, join='outer',
#                                  dim=expt_da)

In [ ]:
with progress.ProgressBar():
    dsets_aligned_ = dask.compute(dsets_aligned)[0]

In [ ]:
source_ids = list(dsets_aligned_.keys())
source_da = xr.DataArray(source_ids, dims='source_id', name='source_id',
                         coords={'source_id': source_ids})

big_ds = xr.concat([ds.reset_coords(drop=True)
                    for ds in dsets_aligned_.values()],
                    dim=source_da)

big_ds

In [ ]:
df_all = big_ds.sel(year=slice(1900, 2100)).to_dataframe().reset_index()
df_all.head()

In [ ]:
sns.relplot(data=df_all,
            x="year", y="tas", hue='experiment_id',
            kind="line", ci="sd", aspect=2);

In [ ]:
print(df_all.shape)
df_by_exp = df_all.groupby('experiment_id')

tas_exp = []
for name, group in df_by_exp:
    group_mon = group.groupby('rawmonth')
    tas = np.empty((13,251))
    tas[0,:]=np.arange(1850,2101)
    month = []
    year = []
    for mon, mongroup in group_mon:
        if mon>251:
            break
        montemp = np.nanmean(mongroup['tas'])
        i_month = int(mon%12)
        i_year = int(np.floor(mon/12))
        tas[i_month,i_year] = montemp
    #print(np.count_nonzero(tas))
    #print(np.count_nonzero(np.isnan(tas)))
    tas_exp.append(tas)
    #years = (np.asarray(months)/12)+1850
    #plt.plot(years,avg_temps,label=name,alpha=0.6)
    #plt.show()

In [ ]:
# print(df_all.shape)
# df_by_exp = df_all.groupby('experiment_id')

# plt.figure(figsize=(8,6))
# for name, group in df_by_exp:
#     group_mon = group.groupby('rawmonth')
#     avg_temps = []
#     months = []
#     for mon, mongroup in group_mon:
#         montemp = np.nanmean(mongroup['tas'])
#         avg_temps.append(montemp)
#         months.append(mon)
#     temps = np.array(group['tas'])
#     print(temps.shape)
#     print(group.shape)
#     years = (np.asarray(months)/12)+1850
#     plt.plot(years,avg_temps,label=name,alpha=0.6)
#     #plt.show()
# #plt.xlim(2000,2020)
# #plt.ylim(270,296)
# plt.legend()

In [ ]:
print(df_all['year'].shape)
#years = np.array(df_all['year'])
#print(years.shape)
plt.figure(figsize=(20,6))
plt.plot(years,'b.')

In [ ]:
plt.figure(figsize=(20,6))
for name, group in df_by_exp:
    for 
    plt.plot(group['year'],label=name)
plt.legend()